# Secondary Index Ver 6.1+

## Key Features: 
### Revised SI layout in memory
 * EE uses Linux Shared RAM.  Allows for Warm Start of SIs.
 * Revised B-Tree model, one B-tree per partition. 
 * One unique 14 byte entry per record indexed.
 * Indexing Bin Data (same as before) - _types_ supported: Numeric, String and Geo2DSphere. (BLOBs have also been added now.)
 * CDTs: Indexing Top level Map and Lists data types. MapKeys, MapValues or ListValues of above _types_ can be indexed.
 * 6.1 extends above top-level only indexing capability of CDTs to any context or nested path in a CDT. (Pre 6.1 only top-level bin data containg a map or list data type could be indexed.)
 
 Partition based layout allows query to progress with partition awareness. Allows users to have fine control over query progress. Maintains query progress through cluster changes. No need to track "failOnClusterChange" events, store and delay consumption of results in clients, and restart query in case of cluster change.
 
 <img src="./graphics/SIOverview.png"
     alt="SIOverview"
     style="center; margin-right: 10px;"
     width="800"
     height="640"/>


## Presentation Outline 
* Connecting to Aerospike One node cluster 
* SI 6.1 Features



#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [51]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

In [52]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>9.0.3</version>
  </dependency>
</dependencies>

####  Add required Java Client Imports

These are some of the Aerospike Java Client imports needed to start developing our Application interactively.  We will add others, as needed, as we develop our solution.

In [53]:
//Require Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


#### Connect to the Aerospike Server
Instantiate the client object. Let us write a record and read it back.
We have a namespace **_test_** pre-defined on the server.

In [54]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Initialized the client and connected to the cluster.


## Secondary Indexes in ver 6.1 for Nested CDTs
Now that we are all setup, let us get back to discussing the new features in ver 6.1 Secondary Indexes.  

## Import Modules
Let us start by importing all the modules we will need for the SI examples.

In [55]:
// import all needed modules
import java.util.ArrayList;
import java.util.Arrays;
import java.util.HashMap;
import java.util.List;
import java.util.Map;

import com.aerospike.client.AerospikeException;
import com.aerospike.client.Key;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.ClientPolicy;
import com.aerospike.client.Record;
import com.aerospike.client.Operation;
import com.aerospike.client.Value;
import com.aerospike.client.Value.ListValue;
import com.aerospike.client.Value.MapValue;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.ListPolicy;
import com.aerospike.client.cdt.ListOrder;
import com.aerospike.client.cdt.ListWriteFlags;
import com.aerospike.client.cdt.MapOperation;
import com.aerospike.client.cdt.MapPolicy;
import com.aerospike.client.cdt.MapOrder;
import com.aerospike.client.cdt.MapWriteFlags;
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.query.RecordSet;
import com.aerospike.client.Record;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.query.IndexType;
import com.aerospike.client.query.IndexCollectionType;
import com.aerospike.client.task.IndexTask;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;
import com.aerospike.client.cdt.CTX;

## Define Constants and Helper Functions
Define constants for the namespaces *test*, set *cdt-indexing*, and helper functions _createIndex_, _dropIndex_, _executeQueryAndPrintResults_, and _truncateTestData_.

In [56]:
final String Namespace = "test";
final String Set = "cdt-indexing";
final String ListBin = "list_bin";
final String MapBin = "map_bin";


// convenience function to create an index - essentially a pass-through to the client API
void createIndex(String idxName, String binName, IndexType idxType, IndexCollectionType colType, CTX... ctx) {
    try {
            IndexTask task = client.createIndex(null,
                                       Namespace,
                                       Set,
                                       idxName,
                                       binName,
                                       idxType,
                                       colType,
                                       ctx);
            task.waitTillComplete(1000, 0);
        }
        catch (AerospikeException ae) {
            if (ae.getResultCode() != ResultCode.INDEX_ALREADY_EXISTS) {
                throw ae;
            }
        } 
        System.out.format("Created index %s on ns=%s set=%s bin=%s.\n", 
                                    idxName, Namespace, Set, binName);
}

// convenience function to drop an index - essentially a pass-through to the client API
void dropIndex(String idxName) {
    try {
        IndexTask task = client.dropIndex(null, Namespace, Set, idxName);
    }
    catch (AerospikeException ae) {
        if (ae.getResultCode() != ResultCode.INDEX_NOTFOUND) {
            throw ae;
        }
    } 
    System.out.format("Dropped index %s.\n", idxName);
}

// convenience function to execute a query using the input filter and print the results
void executeQueryAndPrintResults(Filter filter, String binName) {
    Statement stmt = new Statement();
    stmt.setNamespace(Namespace);
    stmt.setSetName(Set);
    stmt.setFilter(filter);
    stmt.setBinNames(binName);
    RecordSet rs = client.query(null, stmt);
    while (rs.next()) {
        Key key = rs.getKey();
        Record record = rs.getRecord();
        System.out.format("key=%s bins=%s\n", key.userKey, record.bins);
    }
    //System.out.println();
    rs.close();
}

// convenience function to truncate test data
void truncateTestData() {
    try {
        client.truncate(null, Namespace, null, null);
    }
    catch (AerospikeException e) {
        // ignore
    }
}

# Cleanup

In [57]:
%sh asadm --enable -e "manage sindex delete idx_geo ns test set cdt-indexing"

## CDT Test Data
We will illustrate how to create indexes on CDTs and issues queries using two CDTs - a nested List and a nested Map that are populated in 10 records each.

A nested List in records with user_key (id) 1-10. The list is held in bin list_bin and has the following numeric, string, and List elements.



Numeric elements: user_key to user_key+4.
For example, for user_key=1: 1, 2, 3, 4, 5.
String elements: "s"+user_key to "s"+(user_key+4).
For example, for user_key=1: "s1", "s2", "s3", "s4", "s5"
Nested List element: holds numeric elements (10\*user_key)+1 to (10\*user_key)+5.

For example, for user_key=1: [11, 12, 13, 44, 15].
So, the value in list_bin for the record with user_key=1 looks like:

[ 1,"s1", 2, "s2", 3, "s3" 4, "s4", 5, "s5", [ 11, 12, 13, 14, 15 ] ].


![fig_CDTDM_1](./graphics/CDT_DM_1.png)

A nested Map in records with user_key (id) 101-110. The map is held in bin map_bin and has the following numeric, string, and List elements. Below, we have used i with a shorthand for i=user_key-100.


"oid": i.
For example, for i=1: 1.
"obj": a nested map with the following key-value pairs: {"attr1": (10\*i)+1, "attr2": "s"+(10\*i)+2, "subobj": {"attr3": (100\*i)+1, "attr4": (100\*i)+2}}.
For example, for i=1: {"attr1": 11, "attr2": "s12", "subobj": {"attr3": 101, "attr4": 102}}.
i: ["s"+i, "s"+i+1].

For example, for i=1: the key-value pair is 1: ["s1", "s2"].
So, the value in map_bin for the record with user_key=1 looks like:

{
  "oid": 1, 
  "obj": {"attr1": 11, "attr2": "s12", "subobj": {"attr3": 101, "attr4": 102}}, 
  1: ["s1", "s2"]
}

![fig_DM_2](./graphics/CDT_DM_2.png)


## Populate Test Data
Execute the cell below to populate the test data described above.

**We will insert 10 records with userkey=1 through 10 with ListBin, and  records with userkey = 101 thru 110 with MapBin.**


In [58]:
WritePolicy wpolicy = new WritePolicy();
wpolicy.sendKey = true;

// create the list_bin with [i..i+4, "s"+i.."s"+4, [10*i+1..10*i+5]]
for (int user_key = 1; user_key <= 10; user_key++) {
    Key key = new Key(Namespace, Set, user_key);

    // append integer and string elements
    ArrayList<Value> list_val = new ArrayList<Value>();
    for (int n = user_key; n < user_key+5; n++) {
        list_val.add(Value.get(n));
        list_val.add(Value.get("s"+Integer.toString(n)));
    }

    // append string elements
    //for (int n=user_key; n < user_key+5; n++) {
    //    list_val.add(Value.get("s"+Integer.toString(n)));
    //}

    // append the nested list element
    ArrayList<Integer> list_int = new ArrayList<Integer>();
    for (int n=user_key*10+1; n < user_key*10+6; n++) {
        list_int.add(n);
    }
    ListValue list_int_val = new ListValue(list_int);

    // create the list bin
    Record record = client.operate(wpolicy, key,
                                    ListOperation.clear(ListBin), 
                                    ListOperation.appendItems(ListBin, list_val), 
                                    ListOperation.append(ListBin, list_int_val));
}

// create map bin with 
//    "oid": i,
//    "obj": {"attr1": 10*i+1, "attr2": "s"+10*+i+2, "subobj": {"attr3": 100*1+1, "attr4": 100*1+2}}, 
//    i: ["s"+i, "s"+i+1]


for (int user_key = 101; user_key <= 110; user_key++) {
    Integer i = user_key - 100;
    
    Key key = new Key(Namespace, Set, user_key);
    
    // construct obj map value
    HashMap <String, Value> obj = new HashMap <String, Value>();
    obj.put("attr1", Value.get(10*i+1));    
    obj.put("attr2", Value.get("s"+Integer.toString(10*i+2)));   
    HashMap <String, Integer> subobj = new HashMap <String, Integer>();
    subobj.put("attr3", 100*i+1);    
    subobj.put("attr4", 100*i+2);    
    obj.put("subobj", new MapValue(subobj));   

    // construct arr list value
    ArrayList<String> arr = new ArrayList<String>();
    arr.add("s"+Integer.toString(i*10+1));
    arr.add("s"+Integer.toString(i*10+2));
    
    // create the map in map_bin
    MapPolicy mPolicy = new MapPolicy(MapOrder.UNORDERED, MapWriteFlags.DEFAULT);
    Record record = client.operate(wpolicy, key,
                                MapOperation.clear(MapBin), 
                                MapOperation.put(mPolicy, MapBin, 
                                            Value.get("oid"), Value.get(i)),
                                MapOperation.put(mPolicy, MapBin, 
                                            Value.get("obj"), new MapValue(obj)),
                                MapOperation.put(mPolicy, MapBin, 
                                            Value.get(i), new ListValue(arr))
                                ); 
}

## Examine Test Data
View the test data by executing the following commands in a Jupyter terminal tab.

**aql -c "select list_bin from test.cdt-indexing"**
```
+--------------------------------------------------------------------------------------------+----+
| list_bin                                                                                   | PK |
+--------------------------------------------------------------------------------------------+----+
| LIST('[3, "s3", 4, "s4", 5, "s5", 6, "s6", 7, "s7", [31, 32, 33, 34, 35]]')                | 3  |
| LIST('[2, "s2", 3, "s3", 4, "s4", 5, "s5", 6, "s6", [21, 22, 23, 24, 25]]')                | 2  |
| LIST('[10, "s10", 11, "s11", 12, "s12", 13, "s13", 14, "s14", [101, 102, 103, 104, 105]]') | 10 |
| LIST('[9, "s9", 10, "s10", 11, "s11", 12, "s12", 13, "s13", [91, 92, 93, 94, 95]]')        | 9  |
| LIST('[6, "s6", 7, "s7", 8, "s8", 9, "s9", 10, "s10", [61, 62, 63, 64, 65]]')              | 6  |
| LIST('[4, "s4", 5, "s5", 6, "s6", 7, "s7", 8, "s8", [41, 42, 43, 44, 45]]')                | 4  |
| LIST('[8, "s8", 9, "s9", 10, "s10", 11, "s11", 12, "s12", [81, 82, 83, 84, 85]]')          | 8  |
| LIST('[5, "s5", 6, "s6", 7, "s7", 8, "s8", 9, "s9", [51, 52, 53, 54, 55]]')                | 5  |
| LIST('[1, "s1", 2, "s2", 3, "s3", 4, "s4", 5, "s5", [11, 12, 13, 14, 15]]')                | 1  |
| LIST('[7, "s7", 8, "s8", 9, "s9", 10, "s10", 11, "s11", [71, 72, 73, 74, 75]]')            | 7  |
+--------------------------------------------------------------------------------------------+----+
```
![fig_CDTDM_1](./graphics/CDT_DM_1.png)

**aql -c "select map_bin from test.cdt-indexing"**

```
+------------------------------------------------------------------------------------------------------------------+---+
| map_bin                                                                                                          |PK |
+------------------------------------------------------------------------------------------------------------------+---+
|MAP('{9:["s91", "s92"], "obj":{"attr1":91, "attr2":"s92", "subobj":{"attr3":901, "attr4":902}}, "oid":9}')        |109|
|MAP('{1:["s11", "s12"], "obj":{"attr1":11, "attr2":"s12", "subobj":{"attr3":101, "attr4":102}}, "oid":1}')        |101|
|MAP('{2:["s21", "s22"], "obj":{"attr1":21, "attr2":"s22", "subobj":{"attr3":201, "attr4":202}}, "oid":2}')        |102|
|MAP('{5:["s51", "s52"], "obj":{"attr1":51, "attr2":"s52", "subobj":{"attr3":501, "attr4":502}}, "oid":5}')        |105|
|MAP('{10:["s101", "s102"], "obj":{"attr1":101, "attr2":"s102", "subobj":{"attr3":1001, "attr4":1002}}, "oid":10}')|110|
|MAP('{3:["s31", "s32"], "obj":{"attr1":31, "attr2":"s32", "subobj":{"attr3":301, "attr4":302}}, "oid":3}')        |103|
|MAP('{4:["s41", "s42"], "obj":{"attr1":41, "attr2":"s42", "subobj":{"attr3":401, "attr4":402}}, "oid":4}')        |104|
|MAP('{8:["s81", "s82"], "obj":{"attr1":81, "attr2":"s82", "subobj":{"attr3":801, "attr4":802}}, "oid":8}')        |108|
|MAP('{7:["s71", "s72"], "obj":{"attr1":71, "attr2":"s72", "subobj":{"attr3":701, "attr4":702}}, "oid":7}')        |107|
|MAP('{6:["s61", "s62"], "obj":{"attr1":61, "attr2":"s62", "subobj":{"attr3":601, "attr4":602}}, "oid":6}')        |106|
+------------------------------------------------------------------------------------------------------------------+---+
10 rows in set (0.028 secs)
```

![fig_DM_2](./graphics/CDT_DM_2.png)

## Non-Collection Indexes
We will illustrate how indexes on non-collection elements are created and used. We will start with results in mind, and then create the appropriate index, issue a query using the index, and show the results.

### Equality queries
Get records with a specific integer or string value at a specific index, rank, or key position of a List or a Map.

#### Records with a specific value at index X of a List or Map
##### The top level List in list_bin has has numeric value = 3 at List Index = 2.

![fig_EX_1](./graphics/CDT_EX_1.png)


In [59]:
// Create an index for element at position 2 of the top List in list_bin. 
// idx_list_bin_top_pos_2_num
createIndex("idx_list_bin_top_pos_2_num", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.listIndex(2));

// issue the query and print results
System.out.println("Records with the value 3 at position 2 of the top List in list_bin:");
Filter filter = Filter.equal(ListBin, 3, CTX.listIndex(2));
executeQueryAndPrintResults(filter, ListBin);

Created index idx_list_bin_top_pos_2_num on ns=test set=cdt-indexing bin=list_bin.
Records with the value 3 at position 2 of the top List in list_bin:
key=2 bins={list_bin=[2, s2, 3, s3, 4, s4, 5, s5, 6, s6, [21, 22, 23, 24, 25]]}


In [60]:
%sh asadm --enable -e "manage sindex delete idx_list_bin_top_pos_2_num ns test set cdt-indexing"

##### The top level Map in map_bin has numeric value 7 at map index = 2.


![fig_EX_2](./graphics/CDT_EX_2.png)

In [61]:
// Create an index for element at position 2 of the top Map in map_bin. Note, in the key ordered sequence, 
//   "oid" element is in position 2 after the numeric key and "obj".
// idx_map_bin_top_pos_2_num
createIndex("idx_map_bin_top_pos_2_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.mapIndex(2));

// issue the query and print results
System.out.println("Records with the value 7 at position 2 of the top Map in map_bin:");
Filter filter = Filter.equal(MapBin, 7, CTX.mapIndex(2));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_top_pos_2_num on ns=test set=cdt-indexing bin=map_bin.
Records with the value 7 at position 2 of the top Map in map_bin:
key=107 bins={map_bin={obj={attr2=s72, attr1=71, subobj={attr4=702, attr3=701}}, 7=[s71, s72], oid=7}}


In [62]:
%sh asadm --enable -e "manage sindex delete idx_map_bin_top_pos_2_num ns test set cdt-indexing"

### Records with a specific value at rank X of a List or Map
##### The nested List in list_bin has numeric value = 35 in rank -1 (highest value).

![fig_EX_3](./graphics/CDT_EX_3.png)

In [63]:
// Create an index for element at rank -1 of the nested List in list_bin. Note the nested List is at the 
//   last poistion (index -1) in the top List.
// idx_list_bin_nested_rnk_-1_num
createIndex("idx_list_bin_nested_rnk_-1_num", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
            CTX.listIndex(-1), CTX.listRank(-1));

// issue the query and print results
System.out.println("Records with the value 35 in rank -1 (highest value) in the nested List in list_bin:");
Filter filter = Filter.equal(ListBin, 35, CTX.listIndex(-1), CTX.listRank(-1));
executeQueryAndPrintResults(filter, ListBin);

Created index idx_list_bin_nested_rnk_-1_num on ns=test set=cdt-indexing bin=list_bin.
Records with the value 35 in rank -1 (highest value) in the nested List in list_bin:
key=3 bins={list_bin=[3, s3, 4, s4, 5, s5, 6, s6, 7, s7, [31, 32, 33, 34, 35]]}


In [64]:
%sh asadm --enable -e "manage sindex delete idx_list_bin_nested_rnk_-1_num ns test set cdt-indexing"

##### The nested "subobj" Map in map_bin has numeric value = 901 in rank 0 (lowest value in all map values)
![fig_EX_4](./graphics/CDT_EX_4.png)

In [65]:
// Create an index for element at rank 0 (lowest value) in the nested "subobj" Map in map_bin. 
// idx_map_bin_subobj_rnk_0_num
createIndex("idx_map_bin_subobj_rnk_0_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")), CTX.mapRank(0));

// issue the query and print results
System.out.println("Records with the value 901 in rank 0 (lowest value) in the nested \"subobj\" Map in map_bin:");
Filter filter = Filter.equal(MapBin, 901, 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")), CTX.mapRank(0));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_subobj_rnk_0_num on ns=test set=cdt-indexing bin=map_bin.
Records with the value 901 in rank 0 (lowest value) in the nested "subobj" Map in map_bin:
key=109 bins={map_bin={9=[s91, s92], obj={attr2=s92, attr1=91, subobj={attr4=902, attr3=901}}, oid=9}}


In [66]:
%sh asadm --enable -e "manage sindex delete idx_map_bin_subobj_rnk_0_num ns test set cdt-indexing"

##### Records with mapkey value "subobj" Map in map_bin key "obj" ... Index all MapKeys in "obj"
![fig_EX_4](./graphics/CDT_EX_4.png)

In [67]:
// Create an index for mapkeys in for nested map at "obj" Map in map_bin. 

createIndex("idx_map_bin_obj_mapkeys", MapBin, IndexType.STRING, IndexCollectionType.MAPKEYS, 
            CTX.mapKey(Value.get("obj")));

// issue the query and print results
System.out.println("Records with mapkey value \"subobj\" Map in map_bin key \"obj\":");
Filter filter = Filter.contains(MapBin,  IndexCollectionType.MAPKEYS,"subobj",
            CTX.mapKey(Value.get("obj")));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_obj_mapkeys on ns=test set=cdt-indexing bin=map_bin.
Records with mapkey value "subobj" Map in map_bin key "obj":
key=104 bins={map_bin={4=[s41, s42], obj={attr2=s42, attr1=41, subobj={attr4=402, attr3=401}}, oid=4}}
key=102 bins={map_bin={2=[s21, s22], obj={attr2=s22, attr1=21, subobj={attr4=202, attr3=201}}, oid=2}}
key=108 bins={map_bin={8=[s81, s82], obj={attr2=s82, attr1=81, subobj={attr4=802, attr3=801}}, oid=8}}
key=106 bins={map_bin={6=[s61, s62], obj={attr2=s62, attr1=61, subobj={attr4=602, attr3=601}}, oid=6}}
key=109 bins={map_bin={9=[s91, s92], obj={attr2=s92, attr1=91, subobj={attr4=902, attr3=901}}, oid=9}}
key=101 bins={map_bin={1=[s11, s12], obj={attr2=s12, attr1=11, subobj={attr4=102, attr3=101}}, oid=1}}
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}
key=110 bins={map_bin={10=[s101, s102], obj={attr2=s102, attr1=101, subobj={attr4=1002, attr3=1001}}, oid=10}}
key=107 bins={map_bin={obj={

In [68]:
%sh asadm --enable -e "manage sindex delete idx_map_bin_obj_mapkeys ns test set cdt-indexing"

#### Records with a specific value at key X of a Map
##### The top level Map in map_bin has numeric value = 8 at key "oid".
![fig_EX_5](./graphics/CDT_EX_5.png)

In [69]:
// Create an index for element with key "oid" in the top level Map in map_bin. 
// idx_map_bin_top_key_oid_num
createIndex("idx_map_bin_top_key_oid_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
            CTX.mapKey(Value.get("oid")));

// issue the query and print results
System.out.println("Records with the value 8 at key \"oid\" in the top level Map in map_bin:");
Filter filter = Filter.equal(MapBin, 8, 
            CTX.mapKey(Value.get("oid")));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_top_key_oid_num on ns=test set=cdt-indexing bin=map_bin.
Records with the value 8 at key "oid" in the top level Map in map_bin:
key=108 bins={map_bin={8=[s81, s82], obj={attr2=s82, attr1=81, subobj={attr4=802, attr3=801}}, oid=8}}


In [70]:
%sh asadm --enable -e "manage sindex delete idx_map_bin_top_key_oid_num ns test set cdt-indexing"

##### The nested "obj" Map in map_bin has string value = "s42" at map key = "attr2".
![fig_EX_6](./graphics/CDT_EX_6.png)

In [71]:
// Create an index for element with key "attr2" in the nested "obj" Map in map_bin. 
// idx_map_bin_obj_key_attr2_str
createIndex("idx_map_bin_obj_key_attr2_str", MapBin, IndexType.STRING, IndexCollectionType.DEFAULT, 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("attr2")));

// issue the query and print results
System.out.println("Records with the value \"s42\" at key \"attr2\" in the nested \"obj\" Map in map_bin:");
Filter filter = Filter.equal(MapBin, "s42", 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("attr2")));
executeQueryAndPrintResults(filter, MapBin);


Created index idx_map_bin_obj_key_attr2_str on ns=test set=cdt-indexing bin=map_bin.
Records with the value "s42" at key "attr2" in the nested "obj" Map in map_bin:
key=104 bins={map_bin={4=[s41, s42], obj={attr2=s42, attr1=41, subobj={attr4=402, attr3=401}}, oid=4}}


In [72]:
%sh asadm --enable -e "manage sindex delete idx_map_bin_obj_key_attr2_str ns test set cdt-indexing"

### Range Queries
Get records having an integer value within a range at a specific index, rank, or key position of a List or a Map. Range queries are supported on integer values only.

We will use the indexes defined above for the range queries too.

#### Records with value in s specific range at index X rank position of a List or Map
##### The top level List in list_bin numeric value is in range 3-5 at List Index = 2
![fig_EX_7](./graphics/CDT_EX_1.png)

In [73]:
// issue the query and print results
createIndex("idx_list_bin_top_pos_2_num", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
              CTX.listIndex(2)); 

System.out.println("Records with value in range 3-5 at position 2 in the top level List in list_bin:");
Filter filter = Filter.range(ListBin, 3, 5, CTX.listIndex(2));
executeQueryAndPrintResults(filter, ListBin);

Created index idx_list_bin_top_pos_2_num on ns=test set=cdt-indexing bin=list_bin.
Records with value in range 3-5 at position 2 in the top level List in list_bin:
key=3 bins={list_bin=[3, s3, 4, s4, 5, s5, 6, s6, 7, s7, [31, 32, 33, 34, 35]]}
key=4 bins={list_bin=[4, s4, 5, s5, 6, s6, 7, s7, 8, s8, [41, 42, 43, 44, 45]]}
key=2 bins={list_bin=[2, s2, 3, s3, 4, s4, 5, s5, 6, s6, [21, 22, 23, 24, 25]]}


In [74]:
%sh asadm --enable -e "manage sindex delete idx_list_bin_top_pos_2_num ns test set cdt-indexing"

##### The top level Map in map_bin has numeric value in range 5 - 7 at map index = 2.
![fig_EX_8](./graphics/CDT_EX_8.png)

In [75]:
// issue the query and print results
createIndex("idx_map_bin_top_pos_2_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.mapIndex(2));  
 
System.out.println("Records with value in range 5-7 at position 2 of the top Map in map_bin:");
Filter filter = Filter.range(MapBin, 5, 7, CTX.mapIndex(2));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_top_pos_2_num on ns=test set=cdt-indexing bin=map_bin.
Records with value in range 5-7 at position 2 of the top Map in map_bin:
key=107 bins={map_bin={obj={attr2=s72, attr1=71, subobj={attr4=702, attr3=701}}, 7=[s71, s72], oid=7}}
key=106 bins={map_bin={6=[s61, s62], obj={attr2=s62, attr1=61, subobj={attr4=602, attr3=601}}, oid=6}}
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}


In [76]:
%sh asadm --enable -e "manage sindex delete idx_map_bin_top_pos_2_num ns test set cdt-indexing"

#### Records with value in s specific range at rank X of a List or Map
##### The nested List in list_bin has numeric value in range 20 – 50 in rank -1 (highest value).
![fig_EX_9](./graphics/CDT_EX_3.png)

In [77]:
// issue the query and print results
createIndex("idx_list_bin_nested_rnk_-1_num", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
              CTX.listIndex(-1), CTX.listRank(-1));

System.out.println("Records with value in range 20-50 in rank -1 (highest value) in the nested List in list_bin:");
Filter filter = Filter.range(ListBin, 20, 60, CTX.listIndex(-1), CTX.listRank(-1));
executeQueryAndPrintResults(filter, ListBin);

Created index idx_list_bin_nested_rnk_-1_num on ns=test set=cdt-indexing bin=list_bin.
Records with value in range 20-50 in rank -1 (highest value) in the nested List in list_bin:
key=3 bins={list_bin=[3, s3, 4, s4, 5, s5, 6, s6, 7, s7, [31, 32, 33, 34, 35]]}
key=4 bins={list_bin=[4, s4, 5, s5, 6, s6, 7, s7, 8, s8, [41, 42, 43, 44, 45]]}
key=2 bins={list_bin=[2, s2, 3, s3, 4, s4, 5, s5, 6, s6, [21, 22, 23, 24, 25]]}
key=5 bins={list_bin=[5, s5, 6, s6, 7, s7, 8, s8, 9, s9, [51, 52, 53, 54, 55]]}


In [78]:
%sh asadm --enable -e "manage sindex delete idx_list_bin_nested_rnk_-1_num ns test set cdt-indexing"

##### The nested "subobj" Map in map_bin has numeric value in range 500 – 800 in rank 0 (lowest value in all map values).
![fig_EX_10](./graphics/CDT_EX_4.png)

In [79]:
// issue the query and print results
createIndex("idx_map_bin_subobj_rnk_0_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
              CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")), CTX.mapRank(0));  

System.out.println("Records with value in range 500-800 in rank 0 (lowest value) in the nested \"subobj\" Map in map_bin:");
Filter filter = Filter.range(MapBin, 500, 800,
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")), CTX.mapRank(0));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_subobj_rnk_0_num on ns=test set=cdt-indexing bin=map_bin.
Records with value in range 500-800 in rank 0 (lowest value) in the nested "subobj" Map in map_bin:
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}
key=107 bins={map_bin={obj={attr2=s72, attr1=71, subobj={attr4=702, attr3=701}}, 7=[s71, s72], oid=7}}
key=106 bins={map_bin={6=[s61, s62], obj={attr2=s62, attr1=61, subobj={attr4=602, attr3=601}}, oid=6}}


In [80]:
%sh asadm --enable -e "manage sindex delete idx_map_bin_subobj_rnk_0_num ns test set cdt-indexing"

#### Records with value in s specific range at key X of a Map

##### The top level Map in map_bin has numeric value in range 4 - 6 at key "oid".
![fig_EX_11](./graphics/CDT_EX_5.png)


In [81]:
// issue the query and print results
createIndex("idx_map_bin_top_key_oid_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.mapKey(Value.get("oid")));  
 
System.out.println("Records with value in range 4-6 at key \"oid\" in the top level Map in map_bin:");
Filter filter = Filter.range(MapBin, 4, 6, CTX.mapKey(Value.get("oid")));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_top_key_oid_num on ns=test set=cdt-indexing bin=map_bin.
Records with value in range 4-6 at key "oid" in the top level Map in map_bin:
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}
key=104 bins={map_bin={4=[s41, s42], obj={attr2=s42, attr1=41, subobj={attr4=402, attr3=401}}, oid=4}}
key=106 bins={map_bin={6=[s61, s62], obj={attr2=s62, attr1=61, subobj={attr4=602, attr3=601}}, oid=6}}


In [82]:
%sh asadm --enable -e "manage sindex delete idx_map_bin_top_key_oid_num ns test set cdt-indexing"

## Collection Indexes
Collection indexes are defined on List and Map elements with the collection type LIST (list values in the List), MAPKEYS (key values in the Map), or MAPVALUES (values in the Map). The indexes can be used for equality queries on integr and string elements, as well as range queries on integer elements. We will illustrate these variations below.

### Equality Queries
Get records with a specific integer or string value in a List or Map.

#### Records with a List or Map containing a specific value.

##### The nested List in map_bin at MapIndex = 0 contains "s91".
![fig_EX_12](./graphics/CDT_EX_7.png)

In [83]:
// Create an index for the nested List in map_bin. Note, in key-ordered sequence, the numeric key is the first one.
// idx_map_bin_i_list_str
createIndex("idx_map_bin_i_list_str", MapBin, IndexType.STRING, IndexCollectionType.LIST, CTX.mapIndex(0));

// issue the query and print results
System.out.println("Records with the value \"s91\" in the nested list \"arr\" in map_bin:");
Filter filter = Filter.contains(MapBin, IndexCollectionType.LIST, "s91", CTX.mapIndex(0));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_i_list_str on ns=test set=cdt-indexing bin=map_bin.
Records with the value "s91" in the nested list "arr" in map_bin:
key=109 bins={map_bin={9=[s91, s92], obj={attr2=s92, attr1=91, subobj={attr4=902, attr3=901}}, oid=9}}


In [84]:
%sh asadm --enable -e "manage sindex delete idx_map_bin_i_list_str ns test set cdt-indexing"

##### The top level Map in map_bin contains a key whose numeric value = 5.
![fig_EX_13](./graphics/CDT_EX_9.png)

In [85]:
// Create an index for keys in top level Map in map_bin. 
// idx_map_bin_top_mapkeys_num
createIndex("idx_map_bin_top_mapkeys_num", MapBin, IndexType.NUMERIC, IndexCollectionType.MAPKEYS);

// issue the query and print results
System.out.println("Records with key 5 in the top level Map in map_bin:");
Filter filter = Filter.contains(MapBin, IndexCollectionType.MAPKEYS, 5);
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_top_mapkeys_num on ns=test set=cdt-indexing bin=map_bin.
Records with key 5 in the top level Map in map_bin:
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}


In [86]:
%sh asadm --enable -e "manage sindex delete idx_map_bin_top_mapkeys_num ns test set cdt-indexing"

##### The nested "obj" Map in map_bin contains a value "s12".
![fig_EX_14](./graphics/CDT_EX_10.png)

In [87]:
// Create an index for values in nested Map "obj" in map_bin. 
// idx_map_bin_obj_mapvals_str
createIndex("idx_map_bin_obj_mapvals_str", MapBin, IndexType.STRING, IndexCollectionType.MAPVALUES, 
            CTX.mapKey(Value.get("obj")));

// issue the query and print results
System.out.println("Records with the value \"s12\" in the nested Map \"obj\" in map_bin:");
Filter filter = Filter.contains(MapBin, IndexCollectionType.MAPVALUES, "s22", CTX.mapKey(Value.get("obj")));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_obj_mapvals_str on ns=test set=cdt-indexing bin=map_bin.
Records with the value "s12" in the nested Map "obj" in map_bin:
key=102 bins={map_bin={2=[s21, s22], obj={attr2=s22, attr1=21, subobj={attr4=202, attr3=201}}, oid=2}}


In [88]:
%sh asadm --enable -e "manage sindex delete idx_map_bin_obj_mapvals_str ns test set cdt-indexing"

### Range Queries
Get records with a range of integer values in a List or Map.

#### Records with a List or Map containing an integer in a given range.

##### The nested list in list_bin has a value in the numeric value range 85-93.
![fig_EX_15](./graphics/CDT_EX_11.png)

In [89]:
// Create an index for the nested List in map_bin. Note, the nested list is the last element.
// idx_list_bin_nested_list_num
createIndex("idx_list_bin_nested_list_num", ListBin, IndexType.NUMERIC, IndexCollectionType.LIST, 
            CTX.listIndex(-1));

// issue the query and print results
System.out.println("Records with value in range 85-93 in the nested list in list_bin:");
Filter filter = Filter.range(ListBin, IndexCollectionType.LIST, 85, 93, CTX.listIndex(-1));
executeQueryAndPrintResults(filter, ListBin);

Created index idx_list_bin_nested_list_num on ns=test set=cdt-indexing bin=list_bin.
Records with value in range 85-93 in the nested list in list_bin:
key=8 bins={list_bin=[8, s8, 9, s9, 10, s10, 11, s11, 12, s12, [81, 82, 83, 84, 85]]}
key=9 bins={list_bin=[9, s9, 10, s10, 11, s11, 12, s12, 13, s13, [91, 92, 93, 94, 95]]}


In [90]:
%sh asadm --enable -e "manage sindex delete idx_list_bin_nested_list_num ns test set cdt-indexing"

##### The top level Map in map_bin has a MapKey in range 5-7.
![fig_EX_16](./graphics/CDT_EX_9.png)

In [91]:
// use the index created earlier
createIndex("idx_map_bin_top_mapkeys_num", MapBin, IndexType.NUMERIC, IndexCollectionType.MAPKEYS);  

// issue the query and print results
System.out.println("Records with key in range 5-7 in the top level Map in map_bin:");
Filter filter = Filter.range(MapBin, IndexCollectionType.MAPKEYS, 5, 7);
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_top_mapkeys_num on ns=test set=cdt-indexing bin=map_bin.
Records with key in range 5-7 in the top level Map in map_bin:
key=106 bins={map_bin={6=[s61, s62], obj={attr2=s62, attr1=61, subobj={attr4=602, attr3=601}}, oid=6}}
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}
key=107 bins={map_bin={obj={attr2=s72, attr1=71, subobj={attr4=702, attr3=701}}, 7=[s71, s72], oid=7}}


In [92]:
%sh asadm --enable -e "manage sindex delete idx_map_bin_top_mapkeys_num ns test set cdt-indexing"

##### The nested Map "subobj" in map_bin has MapValue(s) in numeric value in range of 300 – 500.
![fig_EX_17](./graphics/CDT_EX_4.png)

In [93]:
// Create an index for values in nested Map "subobj" in map_bin. 
// idx_map_bin_subobj_mapvals_num
createIndex("idx_map_bin_subobj_mapvals_num", MapBin, IndexType.NUMERIC, IndexCollectionType.MAPVALUES, 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")));

// issue the query and print results
System.out.println("Records with value in range 300-500 in the nested Map \"subobj\" in map_bin:");
Filter filter = Filter.range(MapBin, IndexCollectionType.MAPVALUES, 302, 501, 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_subobj_mapvals_num on ns=test set=cdt-indexing bin=map_bin.
Records with value in range 300-500 in the nested Map "subobj" in map_bin:
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}
key=103 bins={map_bin={obj={attr2=s32, attr1=31, subobj={attr4=302, attr3=301}}, 3=[s31, s32], oid=3}}
key=104 bins={map_bin={4=[s41, s42], obj={attr2=s42, attr1=41, subobj={attr4=402, attr3=401}}, oid=4}}


In [94]:
%sh asadm --enable -e "manage sindex delete idx_map_bin_subobj_mapvals_num ns test set cdt-indexing"

### Build Seconday Index on Records with a specific value in the List
##### The List in list_bin has numeric value = 2.

![fig_EX_7](./graphics/CDT_EX_1.png)

In [102]:
// Create an index for listValue in the top List in list_bin. 
// idx_list_bin_value


for(int icase=0;icase<3;icase++){
  dropIndex("idx_list_bin_value");
  Filter filter = null;
  if(icase==0){
    createIndex("idx_list_bin_value", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.listValue(Value.get(2)));
    System.out.println("Records with the value 2 in the list in any position of the top List in list_bin:");
    //Thread.sleep(2000L); //give time (ms) to create index?  Not needed.
    filter = Filter.equal(ListBin, 2, CTX.listValue(Value.get(2)));
  }
  else if(icase==1){
    //Index the record, only if [11,12,13,14,15]... last list item contains the value 31.
    createIndex("idx_list_bin_value", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.listRank(-1), CTX.listValue(Value.get(31)));
    System.out.println("Records with the value 31 in in any position of the list in list at rank -1, in list_bin:");
    //Thread.sleep(2000L);  //give time (ms) to create index?  Not needed.
    filter = Filter.equal(ListBin, 31, CTX.listRank(-1), CTX.listValue(Value.get(31)));
  }
  else if(icase==2){
    ArrayList<Integer> listOfInts = new ArrayList<Integer>();
    listOfInts.add(11);
    listOfInts.add(12);
    listOfInts.add(13);
    listOfInts.add(14);
    listOfInts.add(15);
    ListValue listOfIntsValue = new ListValue(listOfInts);
    createIndex("idx_list_bin_value", ListBin, IndexType.NUMERIC, IndexCollectionType.LIST, CTX.listValue(listOfIntsValue));
    System.out.println("Records with the value 11 in a list = [11,12,13,14,15] (a list value) in any position of the top List in list_bin:");
    //Its indexing records which in their ListBin has this sub-list item [11,12,13,14,15], anywhere in the top list for ListBin.  
    //Thread.sleep(2000L);  //give time (ms) to create index?  Not needed.
    filter = Filter.contains(ListBin, IndexCollectionType.LIST, 11, CTX.listValue(listOfIntsValue));
  }

  executeQueryAndPrintResults(filter, ListBin);
  System.out.println("- - -\n");
}

Dropped index idx_list_bin_value.
Created index idx_list_bin_value on ns=test set=cdt-indexing bin=list_bin.
Records with the value 2 in the list in any position of the top List in list_bin:
key=2 bins={list_bin=[2, s2, 3, s3, 4, s4, 5, s5, 6, s6, [21, 22, 23, 24, 25]]}
key=1 bins={list_bin=[1, s1, 2, s2, 3, s3, 4, s4, 5, s5, [11, 12, 13, 14, 15]]}
- - -

Dropped index idx_list_bin_value.
Created index idx_list_bin_value on ns=test set=cdt-indexing bin=list_bin.
Records with the value 31 in in any position of the list in list at rank -1, in list_bin:
key=3 bins={list_bin=[3, s3, 4, s4, 5, s5, 6, s6, 7, s7, [31, 32, 33, 34, 35]]}
- - -

Dropped index idx_list_bin_value.
Created index idx_list_bin_value on ns=test set=cdt-indexing bin=list_bin.
Records with the value 11 in a list = [11,12,13,14,15] (a list value) in any position of the top List in list_bin:
key=1 bins={list_bin=[1, s1, 2, s2, 3, s3, 4, s4, 5, s5, [11, 12, 13, 14, 15]]}
- - -



In [47]:
dropIndex("idx_list_bin_value");

Dropped index idx_list_bin_value.


## Clean Up


In [48]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"